In [ ]:
!pip install langchain langchain-core langchain-community langchainhub

In [ ]:
from langchain.tools import tool
import requests
import os

@tool
def lookup_domain_availability(domain: str) -> str:
    """This tool takes in one domain name as input. 
    Looks up if domain is registerd at a registrar or not. 
    If its registered the response output will mention the registrar name or else the output will be `available`."""    
    api_url = 'https://api.api-ninjas.com/v1/whois?domain={}'.format(domain)
    response = requests.get(api_url, headers={'X-Api-Key': os.environ['API_NINJAS_API_KEY']})
    if response.status_code == requests.codes.ok:
        response_json = response.json()
        if response_json:
            if 'registrar' in response_json:
                return str(f" ***The domain {domain} is already registered at the registrar : {response_json['registrar']} . It is not available for purchase*** ") 
                  
    return str(f" ***The domain {domain} is available for purchase*** ")
    

@tool
def purchase_domain(domain:str) -> str:
    """This tool takes one domain name and registers it on behalf of the customer"""

    return str(f" ***Domain {domain} has been purchased successfully*** ")

In [ ]:
lookup_domain_availability.run("google.com")

In [ ]:
tools = [lookup_domain_availability,purchase_domain]

In [ ]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions,convert_to_ollama_tool

converted_tools = [convert_to_ollama_tool(tool) for tool in tools]

In [ ]:
from langchain_openai import ChatOpenAI
llm_openai_ollama = ChatOpenAI(
    api_key="ollama",
    model="llama3:70b",
    base_url="https://4nsswzlwu2gg6n-11434.proxy.runpod.net/v1",
)


In [ ]:
llm_openai_ollama = llm_openai_ollama.bind_tools(converted_tools)

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm_openai_ollama, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)

In [ ]:
agent_executor.invoke({"input":" Suggest 2 domain names for my pizza and burger restaurant business and let me know if they are available for purchase ?"})

In [ ]:
agent_executor.invoke({"input":" Please purchase pizzaburgerlove.com and let me know if purchase was successfull ?"})

In [ ]:
agent_executor.invoke({"input":" Please suggest a domain name for my IT services company . do not buy it ?"})

In [ ]:
agent_executor.invoke({"input":" Please suggest 3 domain names for my IT services company. do not check their availability and do not buy them. "})